In [1]:
import re

pd.options.display.max_rows = 9999
pd.options.display.max_columns = 9999

In [2]:
df = pd.read_csv("2019_사업보고서_01_재무상태표_20200526.txt",sep = "\t",  engine='python', encoding = "cp949")

In [3]:
df_A = df[df["업종명"].values == '기초 의약물질 및 생물학적 제제 제조업'].reset_index()

In [4]:
df_B = df[df["업종명"].values == '의약품 제조업'].reset_index()

In [5]:
df_C = df[df["업종명"].values == '의료용품 및 기타 의약 관련제품 제조업'].reset_index()

In [6]:
df_D = df[df["업종명"].values == '자연과학 및 공학 연구개발업'].reset_index()

In [7]:
df_total = pd.concat([df_A, df_B, df_C,df_D]).reset_index()

In [8]:
df_total = df_total[["회사명", "시장구분", "업종명", "결산기준일","항목명", "당기","전기"]]
df_total_kosdaq= df_total[df_total["시장구분"] == "코스닥시장상장법인"]
len(df_total_kosdaq)

5377

In [9]:
a = list(df_total_kosdaq["항목명"].values)
b = []
for i in range(5377):
    b.append(a[i].replace(" ",""))

In [10]:
df_total_kosdaq["항목명"] = b
df_total_kosdaq
df_total_kosdaq.reset_index(drop = True).tail()

,회사명,시장구분,업종명,결산기준일,항목명,당기,전기
5372,헬릭스미스,코스닥시장상장법인,자연과학 및 공학 연구개발업,2019-12-31,기타자본,"4,238,976,442","2,715,840,641"
5373,헬릭스미스,코스닥시장상장법인,자연과학 및 공학 연구개발업,2019-12-31,기타포괄손익누계액,"-9,056,999","-36,666,885"
5374,헬릭스미스,코스닥시장상장법인,자연과학 및 공학 연구개발업,2019-12-31,이익잉여금(결손금),"-220,415,170,404","-113,879,623,895"
5375,헬릭스미스,코스닥시장상장법인,자연과학 및 공학 연구개발업,2019-12-31,자본총계,"198,687,111,539","136,993,925,314"
5376,헬릭스미스,코스닥시장상장법인,자연과학 및 공학 연구개발업,2019-12-31,자본과부채총계,"353,302,234,564","269,313,074,461"


In [11]:
name_list= list(df_total_kosdaq[df_total_kosdaq["항목명"] == "무형자산"]["회사명"])

In [12]:
result = df_total_kosdaq[df_total_kosdaq["회사명"].isin(name_list)].reset_index(drop = True)

In [1]:
screen = ["현금및현금성자산","유형자산","무형자산","자산총계","부채총계","자본총계","자본금"]

In [2]:
fraud_list = ["코오롱티슈진","신라젠"]

In [3]:
result_A = result[result["항목명"].isin(screen)]
result_A["fraud"] = 0
result_A["fraud"][result_A["회사명"].isin(fraud_list)] = 1
result_A.reset_index(drop=True, inplace = True)

NameError: name 'result' is not defined

In [ ]:
result_A["당기"] = result_A["당기"].apply(lambda x: str(x).replace(",","")).astype("float")
result_A["전기"] = result_A["전기"].apply(lambda x: str(x).replace(",","")).astype("float")
result_A["당기/전기"] = result_A["당기"] / result_A["전기"]
result_A.drop([266], inplace = True)
reindex = (range(1,len(result_A)+1))
result_A.reindex(reindex)

In [17]:
df_result = pd.DataFrame(columns = screen)

In [18]:
list_k = list(result_A["당기/전기"].T)

In [19]:
for i in range(1,541):
    if i%6 == 0:
            df_result.loc[i-6] = list_k[i-6:i]
df_result["fraud"] = ""
df_result["회사명"] = list(result_A["회사명"].unique())
df_total = df_result.reset_index(drop=True)
df_total= df_total[["회사명","현금및현금성자산","유형자산","무형자산","자산총계","부채총계","자본총계","fraud"]]
df_total["fraud"] = 0
df_total["fraud"][df_total["회사명"]=="코오롱티슈진"] = 1
df_total["fraud"][df_total["회사명"]=="신라젠"] = 1
df_total

,회사명,현금및현금성자산,유형자산,무형자산,자산총계,부채총계,자본총계,fraud
0,강스템바이오텍,2.127242,1.599060,0.390239,1.490159,1.523076,1.477636,0
1,네이처셀,1.428026,0.924139,1.305322,1.760938,1.151339,1.898698,0
2,녹십자셀,0.503730,1.545771,0.955291,1.223338,1.185090,1.256132,0
3,바이오솔루션,0.544226,2.259070,1.168506,0.979992,0.976058,0.980297,0
4,브릿지바이오,8.516175,5.795180,0.093937,6.812643,0.081933,-1.640743,0
5,셀루메드,0.532378,0.756563,0.184135,0.814964,1.526685,0.165008,0
6,애니젠,0.518121,0.988392,0.986496,0.867541,1.121812,0.700797,0
7,에이비엘바이오,0.746888,1.867913,5.431465,0.846234,0.914979,0.841019,0
8,엔지켐생명과학,0.660676,1.038547,3.052358,1.256728,1.307636,1.249614,0
9,엔케이맥스,19.333842,4.317883,38.432909,2.378998,1.677363,3.041927,0


In [34]:
model = sm.OLS.from_formula("np.log(무형자산) ~ scale(현금및현금성자산)+ scale(np.log(유형자산))+scale(자산총계)+scale(np.log(부채총계))+scale(자본총계)+C(fraud)+0", data=df_total)
result1 = model.fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:           np.log(무형자산)   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.256
Method:                 Least Squares   F-statistic:                     6.057
Date:                Sat, 30 May 2020   Prob (F-statistic):           2.78e-05
Time:                        03:35:41   Log-Likelihood:                -106.99
No. Observations:                  89   AIC:                             228.0
Df Residuals:                      82   BIC:                             245.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
C(fraud)[0]             0.0140    